In [ ]:
import numpy as np
import pylab as plt
import qutip as qt
import ipywidgets

First we define several functions to help with plotting

In [ ]:
def PlotStateDynamics(result, reference_state, *args, **kwargs):
    """
    Plot the evolution of the state given 
        the result of mesolve
    
    Args:
        state (qobj): state to calculate a scalar product with
        result (obj): result object from mesolve

    """

    plt.ylabel(r"$|\langle \psi(t) | 0 \rangle|^2$", fontsize=16)
    plt.xlabel(r'time', fontsize=16)
    plt.plot(result.times, 
             [np.abs(reference_state.overlap(state))**2 for state in result.states],
              *args, **kwargs)

In [ ]:
def PlotExpectationDynamics(result, idx=0, *args, **kwargs):
    """
    Plot the evolution of the expectation value given 
        the result of mesolve
    
    Args:
        result (obj): result object from mesolve
        idx (int): index of value in result.expext

    """

    plt.ylabel(r"Charge $\langle n \rangle$", fontsize=16)
    plt.xlabel(r'time', fontsize=16)
    plt.axhline(0, c='k');
    plt.plot(result.times, result.expect[idx], *args, **kwargs)

In [ ]:
def PlotStatesDistriubtion(result, eigenstates, num=10, tick_label=None):
    """
    Plot the distribution of states using the result of mesolve
    
    Args:
        result (obj): result object from mesolve
        eigenstates (array): eigenstates given by qutip
        num: number of eigenstates to show

    """

    state_numbers = np.arange(num)
    if tick_label is None:
        tick_label=["|"+str(state)+r"$\rangle$" for state in state_numbers]
    states = np.take(eigenstates[1], state_numbers)
    data = [[np.abs(state.overlap(result.states[idx]))**2 
             for state in states] for idx in range(len(result.times))]

    def PlotStates(time):
        plt.bar(state_numbers, data[time], 0.5,
               tick_label=tick_label);
        plt.show()

    interactive_plot = ipywidgets.interactive(PlotStates, time=
                                              ipywidgets.IntSlider(0, 0, len(result.times), 1,
                                                                   continuous_update=False))
    output = interactive_plot.children[-1]
    output.layout.height = '250px'
    return interactive_plot

<br><b><span style="font-size: 25px;">HW4: Quantum computer (15 points)</span></b>

In this homework we will work out an example of simple quantum computer, which consists of two transmons coupled to a resonator.

## Part 1. A single transmon under periodic drive

A transmon qubit can be realised as a SQUID with a large capacitance. A SQUID is used instead of a single Josephson junction to make the energy spectrum of a qubit tunable by external magnetic flux $\phi_\text{ext}$ flowing through the loop. The dependence of the Josephson energy on the flux is described as
$$E_J = (E^{(1)}_J + E^{(2)}_J) \cos \left( \frac{\phi_\text{ext}}{2} \right) \sqrt{1 + d^2 \tan^2 \left( \frac{\phi_\text{ext}}{2}\right)},$$
where $d = \dfrac{E^{(1)}_J - E^{(2)}_J}{E^{(1)}_J + E^{(2)}_J}$ is assymetry coefficient. Also we can denote for convenience $E_{J\Sigma} = E^{(1)}_J + E^{(2)}_J$. Throughout this homework we will use $\hbar = 1$.

The transmon hamiltonian under drive is given by:
$$\hat{H} = \hat{H}_\text{tr} + \hat{H}_d = E_C \hat{n}^2 - E_J \cos \hat{\phi} + \alpha \sin \omega t \hat{n},$$
where $\hat{H}_\text{tr}$ and $\hat{H}_\text{d}$ are transmon and drive hamiltonians respectively.

In [ ]:
class Qubit:
    """A convenient class representing qubit and drive parameters"""
    
    def __init__(self, Ec, Esum, d, alpha, omega, N):
        """
        Args:
            Ec (float): qubit parameter
            Esum (float): sum of Ej parameters in SQUID
            d (float): assymetry coefficient of SQUID
            alpha (float): drive amplitude
            omega (float): drive frequency
            N (int): number of levels in basis

        """

        self.Ec = Ec
        self.Esum = Esum
        self.d = d
        self.alpha = alpha
        self.omega = omega
        self.N = N
        
    def get_Ej(self, phi_ext):
        """
        Args
            phi_ext (float): external flux (phase)
            
        Returns:
            Ej parameter at a given flux

        """

        return self.Esum*np.cos(phi_ext/2)*np.sqrt(1 + self.d**2*np.tan(phi_ext/2)**2)
    
    """We redefine several qutip operators for convenience
       you can just use them as qubit.operator() """
    
    def eye(self):
        return qt.qeye(self.N)
    
    def create(self):
        return qt.create(self.N)
    
    def destroy(self):
        return qt.destroy(self.N)
    
    def num(self):
        return qt.num(self.N)
    
    def tunneling(self):
        return qt.tunneling(2*self.N+1)
    
    def charge(self):
        return qt.charge(self.N)

<b>Problem 1.1 (2.0 points)</b> Construct the hamiltonian of transmon with drive using qutip, solve the evolution problem and plot the corresponding values. The qubit and drive parameters are given below.

In [ ]:
def TransmonQubit(q: Qubit, phi_ext=np.pi):
    """Returns transmon qubit Hamiltonian
    
    Args:
        q (Qubit): qubit parameters
        phi_ext (float): external flux (phase)
        
    Returns:
        qobj: transmon qubit hamiltonian in the charge basis
             
    """
    
    # YOUR CODE HERE

    return Htr

In [ ]:
def Drive(q: Qubit):
    """Return drive Hamiltonian without time dependence
    
    Args:
        alpha (float): drive amplitude

    """

    # YOUR CODE HERE
    
    return Hd

Lets plot the time evolution of ground state under the given drive

In [ ]:
q = Qubit(Ec=1, Esum=50, d=0.7, alpha=20, omega=20, N=30)
Htr = TransmonQubit(q)
Hd = Drive(q)
times = np.linspace(0, 1, 1000)
eigenstates = Htr.eigenstates()
ground_state = eigenstates[1][0]
Heff = [Htr, [Hd, 'np.sin(w*t)']]
result = qt.mesolve([Htr, [Hd, 'np.sin(w*t)']], ground_state,
                    times, [], [q.charge()], args= {'w': q.omega},
                    options=qt.Options(nsteps=1e5, store_states=True))

# if you catch a local variable 'e' referenced before assignment error, 
# unfortunately, you need to restart the jupyter kernel in order
# to make it work again

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
PlotStateDynamics(result, ground_state)
plt.subplot(1, 2, 2)
PlotExpectationDynamics(result)

In [ ]:
# You can move the slider to change the time. Alternatively, you can
# click on slider and use left < and right > keyboard buttons
PlotStatesDistriubtion(result, eigenstates, num=10)

In order to simplify the calculations, we can use the approximation for $\cos \phi$:
$$\hat{H}_\text{tr} = E_c \hat{n}^2 - E_J \left(1 - \frac12 \hat{\phi}^2 + \frac{1}{24} \hat{\phi}^4 \right)$$
Further we use creation and annihilation operators:
$$\hat{n} = i\left( \frac{E_J}{8E_C} \right)^{1/4} (\hat{a}^\dagger - \hat{a}),\quad \hat{\phi} = \left( \frac{E_C}{2E_J} \right)^{1/4} (\hat{a}^\dagger + \hat{a}),$$
for which holds the commutation relationship $[\hat{a}, \hat{a}^\dagger] = 1$.

After substitution we get:
$$\hat{H}_\text{tr} = -\sqrt{\frac{E_C E_J}{8}} (\hat{a}^\dagger - \hat{a})^2 - \left(E_J - \sqrt{\frac{E_C E_J}{8}} (a^\dagger+\hat{a})^2 + \frac{E_C}{48} (\hat{a}^\dagger + \hat{a})^4\right) = $$
$$=\sqrt{\frac{E_C E_J}{2}} (\hat{a}^\dagger \hat{a} + \hat{a} \hat{a}^\dagger) - E_J - \frac{E_C}{48} (\hat{a}^\dagger + \hat{a})^4 = \sqrt{2E_C E_J} \left(\hat{a}^\dagger \hat{a} + \frac12 \right) - E_J - \frac{E_C}{48} (\hat{a}^\dagger + \hat{a})^4$$

We drop the terms with unequal numbers of $\hat{a}$ and $\hat{a}^\dagger$:
$$\hat{H}_\text{tr} = \sqrt{2E_C E_J} \left(\hat{a}^\dagger \hat{a} + \frac12 \right) - E_J - \frac{E_C}{48} (\hat{a}\hat{a}\hat{a}^\dagger\hat{a}^\dagger + \hat{a}^\dagger\hat{a}^\dagger\hat{a}\hat{a} + \hat{a}\hat{a}^\dagger\hat{a}\hat{a}^\dagger + \hat{a}^\dagger\hat{a}\hat{a}^\dagger\hat{a} + \hat{a}^\dagger\hat{a}\hat{a}\hat{a}^\dagger + \hat{a}\hat{a}^\dagger\hat{a}^\dagger\hat{a}) = $$
$$=\sqrt{2E_C E_J} \left(\hat{a}^\dagger \hat{a} + \frac12 \right) - \frac{E_C}{8} (\hat{a}^\dagger\hat{a}\hat{a}^\dagger\hat{a} + \hat{a}^\dagger\hat{a}) - E_J - \frac{E_C}{16}.$$

If we apply the <a href="https://en.wikipedia.org/wiki/Unitary_transformation_(quantum_mechanics)">unitary transformation</a> with operator $\hat{U}$:
$$\hat{U} = \exp(-i\omega t \hat{a}^\dagger a)$$
the hamiltonian transforms into:
$$\hat{H} \to \hat{H}_\text{tr} + i\alpha \left( \frac{E_j}{8E_c} \right)^{1/4} \sin \omega t \hat{U}^\dagger (\hat{a}^\dagger - \hat{a}) \hat{U} - \omega \hat{a}^\dagger \hat{a}$$

<b>Problem 1.2 (3.0 points)</b> Show that
$$\hat{U}^\dagger (\hat{a}^\dagger - \hat{a}) \hat{U} = \exp(i\omega t) \hat{a}^\dagger - \exp(-i\omega t)\hat{a}$$

<b>Solution</b>

Writing the sine in exponential form and getting rid of fast rotating terms we get:
$$\hat{H} \to \hat{H}_\text{tr} + i\alpha \left( \frac{E_j}{8E_c} \right)^{1/4} \sin \omega t (\exp(i\omega t)\hat{a}^\dagger - \exp(-i\omega t) \hat{a})- \omega \hat{a}^\dagger \hat{a} = $$
$$= \hat{H}_\text{tr} + \frac{\alpha}{2} \left( \frac{E_j}{8E_c} \right)^{1/4} (\exp(i\omega t) - \exp(-i\omega t))(\exp(i\omega t)\hat{a}^\dagger - \exp(-i\omega t) \hat{a}) - \omega \hat{a}^\dagger \hat{a} = $$
$$= \hat{H}_\text{tr} - \frac{\alpha}{2} \left( \frac{E_j}{8E_c} \right)^{1/4} (\hat{a}^\dagger + \hat{a}) - \omega \hat{a}^\dagger \hat{a}.$$
Now the hamiltonian is time-independent, so we can simulate the system by applying evolution operator $\exp(-i\hat{H}t)$.

<b>Problem 1.3 (4.0 points)</b> Write the hamiltonians of transmon and drive using rotating wave approximation. Write a function which simulates evolution of the wave function. Don't forget to transform the wave function back to the original frame of reference. Compare your results with the exact simulation. The use of qutip mesolve in this part is prohibited.

In [ ]:
def TransmonQubitRWA(q: Qubit, phi_ext=np.pi):
    """Return transmon qubit Hamiltonian in RWA
    
    Args:
        q (Qubit): qubit parameters
        phi_ext (float): external flux (phase)
        
    Returns:
        qobj: transmon qubit hamiltonian in RWA
             
    """

    # YOUR CODE HERE

    return Htr

In [ ]:
def DriveRWA(q: Qubit, phi_ext=np.pi):
    """Return drive Hamiltonian in RWA
    
    Args:
        q (Qubit): qubit parameters
        phi_ext (float): external flux (phase)
        
    Returns:
        qobj: drive hamiltonian in RWA 

    """

    # YOUR CODE HERE

    return Hd

In [ ]:
def EvolutionSolver(H, initial_state, S, e_ops, times):
    """Given the initial state returns the states of the system 
        at provided times and their expectation values
        
    Args:
        H (qobj): hamiltonian of a system
        S (qobj): auxiliary operator. Unitary operator which we've used for 
            transformation is U = exp(-i*S*t)
        initial_state: (qobj): initial state of a system
        e_ops (array of qobj): array of operators to evalute the expectations 
            at evolved states
        times (array): times for system evolution
        
    Returns:
        result: an object similar to the result of qutip mesolve. It should contain:

            states (array): array of states corresponding 
                to the given times
            expect (array): list of expectation values of provided operators
            times: (array): the times given to the solver
            
    .. hints::

        You can use obj.expm() from qutip to calculate matrix exponent
        Also, don't forget that exp(A)exp(B) is not always equal to exp(A+B)

    """
    
    # YOUR CODE HERE

    return type("",(),{"states": states,
                       "expect": [qt.expect(op, states) for op in e_ops],
                       "times": times})()

In [ ]:
times = np.linspace(0, 1, 1000)
Htr_rwa = TransmonQubitRWA(q)
Hd_rwa = DriveRWA(q)

# Note: here we write a charge operator in the energy basis
charge_rwa = 1j*(q.get_Ej(np.pi)/(8*q.Ec))**0.25*(q.create()-q.destroy())
S = q.omega*q.num()

eigenstates_rwa = Htr_rwa.eigenstates()
ground_state_rwa = eigenstates_rwa[1][0]
result_rwa = EvolutionSolver(Htr_rwa + Hd_rwa, ground_state_rwa, S, [charge_rwa], times)

Lets compare the results with exact solution (don't worry if the match is not really accurate):

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
PlotStateDynamics(result_rwa, ground_state_rwa, label="RWA")
PlotStateDynamics(result, ground_state, label="exact")
plt.legend();
plt.subplot(1, 2, 2)
PlotExpectationDynamics(result_rwa, label="RWA")
PlotExpectationDynamics(result, label="exact")
plt.legend();

In [ ]:
PlotStatesDistriubtion(result_rwa, eigenstates_rwa, num=10)

## Part 2. A system of two transmons

A Hamiltonian for a system of two transmons with the drive can be written as:
$$\hat{H} = \hat{H}_\text{tr}^{(1)} + \hat{H}_\text{tr}^{(2)} + \hat{H}_\text{int} + \hat{H}_\text{d},$$

where $\hat{H}_\text{tr}$, $\hat{H}_\text{d}$ are transmon and drive hamiltonians and $\hat{H}_\text{int}$ represents an interaction between transmons.

We have already defined the operators for transmons in the previous part. The interaction term can be written as:

$$\hat{H}_\text{int} = J \hat{n}_1 \hat{n}_2.$$

The drive hamiltonian should also be modified to act on two qubits resulting in:

$$\hat{H}_\text{d} = \alpha (\hat{n}_1 + \hat{n}_2) \sin \omega t.$$

If we apply the unitary transformation with a modified operator:
$$\hat{U} = \exp(-i\omega t (\hat{a}_1^\dagger \hat{a}_1 + \hat{a}_2^\dagger \hat{a}_2)$$
the hamiltonian transforms into
$$\hat{H} \to \hat{H}_\text{tr}^{(1)} + \hat{H}_\text{tr}^{(2)} + \hat{U}^\dagger \hat{H}_\text{int} \hat{U} + \hat{U}^\dagger \hat{H}_\text{d} \hat{U} = $$

$$ = \hat{H}_\text{tr}^{(1)} + \hat{H}_\text{tr}^{(2)} + \hat{U}^\dagger \hat{H}_\text{int} \hat{U} + \frac{\alpha}{2} \left\{ \left( \frac{E^{(1)}_J}{8E^{(1)}_C} \right)^{1/4} (\hat{a}_1^\dagger + \hat{a}_1) + \left( \frac{E^{(2)}_J}{8E^{(2)}_C} \right)^{1/4} (\hat{a}_2^\dagger + \hat{a}_2) \right\} - \omega (\hat{a}_1^\dagger \hat{a}_1 + \hat{a}_2^\dagger \hat{a}_2).$$

<b>Problem 2.1 (2.0 points)</b> Derive the expression for $\hat{U}^\dagger \hat{H}_\text{int} \hat{U}$ under rotating wave approximation.

<b>Solution</b>

<b>Problem 2.2 (4.0 points)</b> Plot the energy levels of a system of two transmon qubits as a function of the coupling parameter $J$. Qubits parameters are given below. Note, that $\phi_\text{ext}$ has different signs for qubits.

In [ ]:
def TwoTransmonQubitsRWA(q1: Qubit, q2: Qubit, J, phi_ext=np.pi):
    """Returns Hamiltonian of two transmons with interaction in RWA
    
    Args:
        q1 (Qubit): qubit 1 parameters
        q2 (Qubit): qubit 2 parameters
        J (float): interaction coefficient between qubits 1 and 2
        phi_ext (float): external flux (phase)
        
    Returns:
        qobj: hamiltonian of two transmon qubits in RWA
             
    """

    # YOUR CODE HERE

    return Htr + Hint

In [ ]:
def TwoTransmonsDriveRWA(q1: Qubit, q2: Qubit, phi_ext=np.pi):
    """Return drive Hamiltonian in RWA
    
    Args:
        q1 (Qubit): qubit 1 parameters
        q2 (Qubit): qubit 2 parameters
        phi_ext (float): external flux (phase)

    """

    # YOUR CODE HERE

    return Hd

In [ ]:
def PlotTwoTransmonQubitsVals(eigenstates, Js, nfirst=8):
    """Plots the energy spectrum
    
    Args:
        Js (array): values of coupling parameters
        eigenstates (array): qubit eigenstates
        nfirst (int): number of energy levels to plot

    """

    plt.xlabel(r"$J$", fontsize=12)
    plt.ylabel(r"Energy, $E$", fontsize=12)
    for state in eigenstates[:, 0, :nfirst].T:
        plt.plot(Js, state)

In [ ]:
q1 = Qubit(Esum=50, d=0.7, Ec=1, alpha=20, omega=20, N=10)
q2 = Qubit(Esum=50, d=0.7, Ec=1, alpha=20, omega=20, N=10)

In [ ]:
Js = np.linspace(0, 2, 100)
eigenstates = np.array([(TwoTransmonsQubitRWA(q1, q2, J=J_, phi_ext=np.pi) + \
                         TwoTransmonsDriveRWA(q1, q2, phi_ext=np.pi)).eigenstates()
                         for J_ in Js])

In [ ]:
PlotTwoTransmonQubitsVals(eigenstates, Js)